In [3]:
import os
import glob
import fnmatch

# data and compute libraries
import pandas as pd

# time-based libraries
from datetime import datetime

# graphing libraries
import matplotlib.pyplot as plt

# other visuals
import squarify

# libraries for printing markdown tables

# mapping libraries

In [4]:
# project home
project_directory = "/home/martinsawojide/Desktop/Spring 25/Mobile Big Data Analytics and Mgt/Project/cpi_project"

# inflation datafiles
inflation_data_directory = f"{project_directory}/inflation_data"

# file of interest: NBS 2024 CPI Report (Dec 2024)
cpi_2024 = './cpi_1NewDec2024.xlsx'

In [5]:
# finding relevant CPI files

def find_relevant_files(month, year, directory):

    # Convert month and year to lowercase for case-insensitive matching
    month = month.lower()
    year = year.lower()

    # Search for files with .xlsx or .xls extensions containing the month and year in any order
    search_patterns = [
        f"*{year}*{month}*.xlsx",
        f"*{month}*{year}*.xlsx",
        f"*{year}*{month}*.xls",
        f"*{month}*{year}*.xls",
    ]

    matching_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            for pattern in search_patterns:
                if fnmatch.fnmatch(file.lower(), pattern.lower()):  # Case-insensitive matching
                    matching_files.append(os.path.join(root, file))
    
    if len(matching_files) > 0:
        print(f"{matching_files}")
        return matching_files[0]                # return the first matching file
    else: raise ValueError(f'no matching file')

In [6]:
# utlity function for cleaning up and filling N/A values

def ffill_dropna(df):

    df.ffill(inplace=True)
    df.dropna(inplace=True)
    

In [7]:
# Converts a month string to its short format (%b).

def format_month_to_short(month_str):
    try:
        # Handle both long and short month formats
        if len(month_str) == 3:  # handling special format issues with 3-letter months
            return datetime.strptime(month_str, "%b").strftime("%b")
        else:  # Convert long format to short format
            return datetime.strptime(month_str, "%B").strftime("%b")
    except ValueError:
        raise ValueError(f"Invalid month string: {month_str}")

In [8]:
# Converts a year string or integer to a standard 4-digit year format (%Y).

def format_year_to_standard(year):
    try:
        # Convert to string if it's an integer
        year_str = str(year)
        # Parse and format the year
        return datetime.strptime(year_str, "%Y").strftime("%Y")
    except ValueError:
        raise ValueError(f"Invalid year value: {year}")

In [9]:
# utility function for maintaing a single format for months

def format_month_inplace(df, month_column="Month"):
    df[month_column] = df[month_column].apply(lambda row: format_month_to_short(row))



In [10]:
# get NBS CPI data (start till Dec 2024)

def get_cluster_cpi_data(report=cpi_2024, cluster_type="composite", data_type="items"):

    cluster_type_options = ["composite", "urban", "rural"]
    data_type_options = ["items", "aggregate", "aggregate_ratios"]

    if cluster_type not in cluster_type_options or data_type not in data_type_options:
        return ValueError(f'USAGE: the acceptable arguments are:\ncluster type: {cluster_type_options}\ndata type: {data_type_options}')

    sheet_name = {
                    "composite": 'Table2',
                    "urban": 'Table3',
                    "rural": 'Table4',
                }

    cpi_data_df = pd.read_excel(report, sheet_name=sheet_name[cluster_type], skiprows=1)
    cpi_data_df.rename(columns={
        'Unnamed: 0': 'Year',
        'Unnamed: 1': 'Month',
    }, inplace=True)
    
    # Fill missing values with the previous numeric values
    ffill_dropna(cpi_data_df)

    # Format month column
    format_month_inplace(df=cpi_data_df)

    # create and return different slices of dataframe based on arguments
    if data_type == "items":
        return cpi_data_df.iloc[:, [0, 1, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5]]
    elif data_type == "aggregate":
        return cpi_data_df.iloc[:, [0, 1, 3, 4, 2]]
    elif data_type == "aggregate_ratios":
        # rename aggregate ratios
        cpi_data_df.rename(columns={
            'Month-on (%)': 'Month-on-Month Change (%)',
            'Year-on (%)': 'Year-on-Year Change (%)',
            '12-month average (%)': '12-Month Average Change (%)',
        }, inplace=True)
        
        return cpi_data_df.iloc[:, [0, 1, -4, -2, -3]]


    

In [22]:
# function that get the cluster type and datatype for get_cluster_cpi_data
# and modifies the dataframe so that the Month column is reformatted to %Y-%m
# using the value in the Year and Month columns and returns the modified dataframe
# the modified dataframe should be saved as a csv file using the cluster type and datatype
def get_cluster_cpi_data_and_save(report=cpi_2024, cluster_type="composite", data_type="items"):

    # Get the CPI data
    cpi_data_df = get_cluster_cpi_data(report=report, cluster_type=cluster_type, data_type=data_type)

    # Format the Month column to YYYY-MM format
    cpi_data_df['Month'] = cpi_data_df.apply(lambda row: f'{row["Year"]}-{datetime.strptime(row["Month"], "%b").strftime("%m")}', axis=1)

    # Drop the Year column
    cpi_data_df.drop(columns=['Year'], inplace=True)

    # Save the modified DataFrame to a CSV file
    csv_filename = f"{cluster_type}_{data_type}.csv"
    cpi_data_df.to_csv(csv_filename, index=False)
    
    return cpi_data_df

In [23]:
get_cluster_cpi_data_and_save(cluster_type="composite", data_type="items")
get_cluster_cpi_data_and_save(cluster_type="composite", data_type="aggregate")
get_cluster_cpi_data_and_save(cluster_type="composite", data_type="aggregate_ratios")

get_cluster_cpi_data_and_save(cluster_type="urban", data_type="items")
get_cluster_cpi_data_and_save(cluster_type="urban", data_type="aggregate")
get_cluster_cpi_data_and_save(cluster_type="urban", data_type="aggregate_ratios")

get_cluster_cpi_data_and_save(cluster_type="rural", data_type="items")
get_cluster_cpi_data_and_save(cluster_type="rural", data_type="aggregate")
get_cluster_cpi_data_and_save(cluster_type="rural", data_type="aggregate_ratios")

,Month,Month-on-Month Change (%),12-Month Average Change (%),Year-on-Year Change (%)
25,1996-12,-2.136666,29.188010,13.661808
26,1997-01,0.739903,26.572758,13.830411
27,1997-02,1.122251,24.219905,12.990504
28,1997-03,4.362095,22.240955,15.202714
29,1997-04,0.562502,20.839353,14.316471
...,...,...,...,...
357,2023-08,2.056391,29.321632,29.949062
358,2023-09,2.385814,29.758015,30.491609
359,2023-10,2.529049,30.238759,31.590193
360,2023-11,2.513104,30.708387,32.267828


In [15]:
# get composite price index values (index and ratios)

def get_composite_cpi_data(report=cpi_2024, data_type="cpi", index_type="all_index"):

    data_type_options = ["cpi", "ratio"]
    index_type_options = ["all_index", "food", "all_less_farm_produce_and_energy"]

    if index_type not in index_type_options or data_type not in data_type_options:
        return ValueError(f'USAGE: the acceptable arguments are:\nindex type: {index_type_options}\ndata type: {data_type_options}')

    cpi_data_df = pd.read_excel(report, sheet_name='Table1', skiprows=2)

    # get slices of the dataframe based on index type
    if index_type == "all_index":
        full_df = cpi_data_df.iloc[2:, :7]
        full_df.rename(columns={
            'Unnamed: 0': 'Year',
            'Unnamed: 1': 'Month',
            'All Items Index': 'Monthly',
            'Unnamed: 3': '12-Month Average',
            'Unnamed: 4': 'Month-on-Month Change (%)',
            'Unnamed: 5': 'Year-on-Year Change (%)',
            'Unnamed: 6': '12-Month Average Change (%)',
        }, inplace=True)

        ffill_dropna(full_df)
        format_month_inplace(df=full_df)
        
        if data_type == "cpi":
            return full_df.iloc[:, 0:4]
        elif data_type == "ratio":
            return full_df.iloc[:, [0, 1, 4, 5, 6]]

    elif index_type == "food":
        full_df = cpi_data_df.iloc[2:, [0, 1, 7, 8, 9, 10, 11 ]]
        full_df.rename(columns={
            'Unnamed: 0': 'Year',
            'Unnamed: 1': 'Month',
            'Food ': 'Monthly',
            'Unnamed: 8': '12-Month Average',
            'Unnamed: 9': 'Month-on-Month Change (%)',
            'Unnamed: 10': 'Year-on-Year Change (%)',
            'Unnamed: 11': '12-Month Average Change (%)',
        }, inplace=True)

        ffill_dropna(full_df)
        format_month_inplace(df=full_df)
        
        if data_type == "cpi":
            return full_df.iloc[:, 0:4]
        elif data_type == "ratio":
            return full_df.iloc[:, [0, 1, 4, 5, 6]]

    elif index_type == "all_less_farm_produce_and_energy":
        full_df = cpi_data_df.iloc[2:, [0, 1, -7, -6, -5, -4, -3 ]]
        full_df.rename(columns={
            'Unnamed: 0': 'Year',
            'Unnamed: 1': 'Month',
            'All Items less Farm Produce and Energy': 'Monthly',
            'Unnamed: 13': '12-Month Average',
            'Unnamed: 14': 'Month-on-Month Change (%)',
            'Unnamed: 15': 'Year-on-Year Change (%)',
            'Unnamed: 16': '12-Month Average Change (%)',
        }, inplace=True)

        ffill_dropna(full_df)
        format_month_inplace(df=full_df)
        
        if data_type == "cpi":
            return full_df.iloc[:, 0:4]
        elif data_type == "ratio":
            return full_df.iloc[:, [0, 1, 4, 6, 5]]
    

In [20]:
# similar function for composite using index_type and data_type
def get_composite_cpi_data_and_save(report=cpi_2024, data_type="cpi", index_type="all_index"):

    # Get the CPI data
    cpi_data_df = get_composite_cpi_data(report=report, data_type=data_type, index_type=index_type)

    # Format the Month column to YYYY-MM format
    cpi_data_df['Month'] = cpi_data_df.apply(lambda row: f'{row["Year"]}-{datetime.strptime(row["Month"], "%b").strftime("%m")}', axis=1)

    # Drop the Year column
    cpi_data_df.drop(columns=['Year'], inplace=True)

    # Save the modified DataFrame to a CSV file
    csv_filename = f"{index_type}_{data_type}.csv"
    cpi_data_df.to_csv(csv_filename, index=False)
    
    return cpi_data_df

In [21]:
get_composite_cpi_data_and_save(index_type="all_less_farm_produce_and_energy", data_type="ratio")
get_composite_cpi_data_and_save(index_type="all_less_farm_produce_and_energy", data_type="cpi")

get_composite_cpi_data_and_save(index_type="all_index", data_type="ratio")
get_composite_cpi_data_and_save(index_type="all_index", data_type="cpi")

get_composite_cpi_data_and_save(index_type="food", data_type="ratio")
get_composite_cpi_data_and_save(index_type="food", data_type="cpi")


/tmp/ipykernel_2855525/1970729029.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.ffill(inplace=True)
/tmp/ipykernel_2855525/1970729029.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.ffill(inplace=True)
/tmp/ipykernel_2855525/1970729029.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.ffill(inplace=True)
/tmp/ipykerne

,Month,Monthly,12-Month Average
25,1996-12,31.788201,30.495332
26,1997-01,31.692665,30.765905
27,1997-02,31.601364,30.978963
28,1997-03,32.557878,31.226202
29,1997-04,32.449912,31.441309
...,...,...,...
357,2023-08,989.693586,859.039570
358,2023-09,1015.805007,882.247298
359,2023-10,1045.644539,906.766869
360,2023-11,1076.837352,932.371570


In [12]:
# get price basket

def get_market_basket(report=cpi_2024, cluster_type="composite", weight_type="items"):

    weight_type_options = ["items", "aggregate"]
    cluster_type_options = ["composite", "urban", "rural"]

    if cluster_type not in cluster_type_options or weight_type not in weight_type_options:
        return ValueError(f'USAGE: the acceptable arguments are:\ncluster type: {cluster_type_options}\nweight type: {weight_type_options}')
    
    sheet_name = {
                "composite": 'Table2',
                "urban": 'Table3',
                "rural": 'Table4',
                }

    if cluster_type == "rural": market_basket_df = pd.read_excel(report, sheet_name=sheet_name[cluster_type], skiprows=1).iloc[1:2, 2:-4]
    else: market_basket_df = pd.read_excel(report, sheet_name=sheet_name[cluster_type], skiprows=1).iloc[:1, 2:-4]

    if weight_type == "aggregate":
        return market_basket_df.iloc[:, [1, 2, 3, 0]].to_dict(orient='list')
    elif weight_type == "items":
        return market_basket_df.iloc[:, [-14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0]].to_dict(orient='list')

In [13]:
get_market_basket()

{'Imported Food': [132.8786430845212],
 'Food': [507.0588900812485],
 'Food &  Non Alcoholic Bev.': [518.0035159781029],
 'Alcoholic Beverage. Tobacco and Kola': [10.870023422874887],
 'Clothing and Footwear': [76.50177479764076],
 'Housing Water, Electricity. Gas and Other  Fuel': [167.3417670918405],
 'Furnishings & Household Equipment Maintenance.': [50.30429132112485],
 'Health.': [30.041719595344983],
 'Transport': [65.08383943029551],
 'Communication': [6.798146513600633],
 'Recreation & Culture.': [6.8692556920957415],
 'Education': [39.43545584224773],
 'Restaurant &  Hotels': [12.116896655754124],
 'Miscellaneous Goods & Services': [16.633313659077338],
 'All Items': [999.9999999999999]}

In [14]:
get_market_basket(cluster_type="rural")

{'Imported Food': [114.03346558957918],
 'Food': [532.2501623514466],
 'Food &  Non Alcoholic Bev.': [540.8933408435455],
 'Alcoholic Beverage. Tobacco and Kola': [10.606303106553293],
 'Clothing and Footwear': [85.68337715684741],
 'Housing Water, Electricity. Gas and Other  Fuel': [147.16440266807825],
 'Furnishings & Household Equipment Maintenance.': [50.72011838442036],
 'Health.': [32.24620846837167],
 'Transport': [57.18640135048233],
 'Communication': [4.005931720723357],
 'Recreation & Culture.': [6.646615393168104],
 'Education': [38.925348101183964],
 'Restaurant &  Hotels': [10.139844568892057],
 'Miscellaneous Goods & Services': [15.7821082377335],
 'All Items': [999.9999999999999]}

In [15]:
get_market_basket(cluster_type="urban")

{'Imported Food': [155.45143810593527],
 'Food': [476.8847287905714],
 'Food &  Non Alcoholic Bev.': [490.58603344696826],
 'Alcoholic Beverage. Tobacco and Kola': [11.185908197369981],
 'Clothing and Footwear': [65.50403131243718],
 'Housing Water, Electricity. Gas and Other  Fuel': [191.5102585444788],
 'Furnishings & Household Equipment Maintenance.': [49.80621275080386],
 'Health.': [27.401177978203123],
 'Transport': [74.5434081193025],
 'Communication': [10.142667529244841],
 'Recreation & Culture.': [7.135934731470604],
 'Education': [40.046464015609814],
 'Restaurant &  Hotels': [14.485013990566927],
 'Miscellaneous Goods & Services': [17.652889383544117],
 'All Items': [999.9999999999998]}

In [16]:
get_market_basket(weight_type="aggregate")

{'All Items less Farm Produce. ': [513.1043122747834],
 'All Items less Farm Produce. and Energy': [405.55493225171676],
 'Imported Food': [132.8786430845212],
 'All Items': [999.9999999999999]}

In [17]:
get_market_basket(cluster_type="rural", weight_type="aggregate")

{'All Items less Farm Produce. ': [480.0942188447083],
 'All Items less Farm Produce. and Energy': [394.6058417257211],
 'Imported Food': [114.03346558957918],
 'All Items': [999.9999999999999]}

In [18]:
get_market_basket(cluster_type="urban", weight_type="aggregate")

{'All Items less Farm Produce. ': [552.6438747349832],
 'All Items less Farm Produce. and Energy': [418.66977694768946],
 'Imported Food': [155.45143810593527],
 'All Items': [999.9999999999998]}

In [19]:
# Obtain state-wise cpi data

def state_wise_cpi_data(year="2024", month="Dec"):

    year = format_year_to_standard(year)                            # format year
    month = format_month_to_short(month)                            # format month

    print(f'Month: {month} Year: {year}')

    # find file relevant to month and year provided and return name
    relevant_file = find_relevant_files(month=month, year=year, directory=inflation_data_directory)

    # load file as dataframe and clean up
    state_cpi_df = pd.read_excel(relevant_file, sheet_name="Table-5", skiprows=8)
    clean_state_cpi_df = state_cpi_df.iloc[:, 1:].dropna()

    cpi_index = 5

    # get column that matches provided month and year
    for idx, val in enumerate(clean_state_cpi_df.columns.tolist()):
        if isinstance(val, datetime):
            month_year_format = val.strftime("%b-%Y")
            if month.lower() in month_year_format.lower() and year in month_year_format:
                cpi_index = idx; break;
    
    state_cpi_current_df = clean_state_cpi_df.iloc[1:, [0, cpi_index, cpi_index+1]].copy()

    state_cpi_current_df.columns = ['State', 'Food_Index', 'All_Items_Index']

    return state_cpi_current_df
        

In [20]:
state_wise_cpi_data(month='Dec', year=2024)

Month: Dec Year: 2024
['/home/martinsawojide/Desktop/Spring 25/Mobile Big Data Analytics and Mgt/Project/cpi_project/inflation_data/ground_truth/2024/cpi_1NewDec2024.xlsx']


,State,Food_Index,All_Items_Index
2,Abia,1139.1,892.2
3,Abuja,1064.5,834.55
4,Adamawa,1048.31,829.18
5,Akwa Ibom,1116.42,876.64
6,Anambra,1093.06,917.98
7,Bauchi,917.98,1120.59
8,Bayelsa,1137.81,889.43
9,Benue,1086.59,819.32
10,Borno,1063.78,763.72
11,Cross River,1173.17,812.66


In [22]:
ng_states = state_wise_cpi_data(month='Dec', year=2024)['State'].to_list()
print(ng_states)

Month: Dec Year: 2024
['/home/martinsawojide/Desktop/Spring 25/Mobile Big Data Analytics and Mgt/Project/cpi_project/inflation_data/ground_truth/2024/cpi_1NewDec2024.xlsx']
['Abia', 'Abuja', 'Adamawa', 'Akwa Ibom', 'Anambra', 'Bauchi', 'Bayelsa', 'Benue', 'Borno', 'Cross River', 'Delta', 'Ebonyi', 'Edo', 'Ekiti', 'Enugu', 'Gombe', 'Imo', 'Jigawa', 'Kaduna', 'Kano', 'Katsina', 'Kebbi', 'Kogi', 'Kwara', 'Lagos', 'Nassarawa', 'Niger', 'Ogun', 'Ondo', 'Osun', 'Oyo', 'Plateau', 'Rivers', 'Sokoto', 'Taraba', 'Yobe', 'Zamfara']
